# Code for generating SLFs for 4 structural components

Story Loss Function Generator for Main Structural Components (SLF_Struc)
========================================================================

Author: Shiva Baddipalli
Institution: Utah State University, USA
Email: shivalinga.baddipalli@usu.edu
Last Updated: June 25, 2025

This script generates Story Loss Functions for four main structural components:
- Shear tabs, RBS one-sided connections, RBS two-sided connections, Slabs

Reference:
----------
Shahnazaryan, D., O'Reilly, G.J., and Monteiro, R. (2021). "Story loss functions 
for seismic design and assessment: Development of tools and application." 
Earthquake Spectra, 37(4), 2813-2839.

SLF Generation Tool: storeyloss Python package
GitHub: https://github.com/davitshahnazaryan3/SLFGenerator

INPUTS REQUIRED:
---------------
1. CSV Files (Component Inventory):
   - File format: "My_Inventory_CASE{case}.csv" where case = 1-9
   - Location: "MyInventory_Structural/MyInventory_Structural_4comp/"
   - Required columns: Component ID, EDP, Component Group, Quantity, Damage States,
     DS1-DS5 (Median Demand), Fragility parameters, Repair costs

2. Components Analyzed:
   - A3: Shear tab connections
   - A4: RBS (Reduced Beam Section) one-sided connections  
   - A5: RBS two-sided connections
   - A6: Slab elements

3. Analysis Parameters:
   - EDP Range: 0-300% inter-story drift ratio, 0.001 increment
   - Correlation Model: 'psd' (peak inter-story drift)
   - Monte Carlo Realizations: 20,000

OUTPUTS GENERATED:
-----------------
- JSON files: output_{case}.json (individual case loss functions)
- CSV file: CombinedOutput_SLF_Struct_4comps.csv (consolidated all cases)

HOW STOREYLOSS LIBRARY IS USED:
-------------------------------
1. storeyloss.SLF() - Initializes the SLF model:
   - component_data: DataFrame with component inventory (from CSV)
   - 'psd': Correlation model for inter-component dependencies
   - edp_range: Array of engineering demand parameter values
   - do_grouping=True: Groups similar components for efficiency
   - realizations=20000: Monte Carlo simulations for uncertainty

2. model.generate_slfs() - Executes SLF generation:
   - Performs Monte Carlo simulations across EDP range
   - Calculates damage probabilities using component fragility functions
   - Estimates repair costs based on damage states and quantities
   - Aggregates component losses to story-level functions

3. model.export_to_json() - Exports results:
   - Saves loss functions as EDP vs. loss relationships
   - Includes statistical measures and component contributions

In [ ]:
import pandas as pd
import json
import storeyloss
import numpy as np

# Configuration
cases = [1, 2, 3, 4, 5, 6, 7, 8, 9]
base_path = "MyInventory_Structural/MyInventory_Structural_4comp/"

# Initialize DataFrame for combined results
combined_df = pd.DataFrame()

# Process each case
for case in cases:
    print(f"Processing Case {case}...")
    
    # Load component inventory data
    component_data = pd.read_csv(f"{base_path}My_Inventory_CASE{case}.csv")
    
    # Define EDP range (0% to 300% inter-story drift ratio)
    edp_range = np.arange(0, 3.001, 0.001)
    
    # Initialize SLF model
    model = storeyloss.SLF(
        component_data,
        'psd',                    
        edp_range=edp_range,
        do_grouping=True,         # Enable component grouping
        realizations=20000        # Monte Carlo simulations
    )
    
    # Generate story loss functions
    out = model.generate_slfs()
    
    # Export to JSON
    output_json_path = f"{base_path}output_{case}.json"
    model.export_to_json(out, output_json_path)
    
    # Load JSON data for combination
    with open(output_json_path, 'r') as file:
        data = json.load(file)
    
    # Extract EDP range and SLF data
    edp_range = data['1']['edp_range']
    slf = data['1']['slf']
    
    # Set EDP range column (only for first case)
    if case == 1:
        combined_df['EDP Range'] = edp_range
    
    # Add SLF data for current case
    combined_df[f'SLF Case {case}'] = slf
    
    print(f"Case {case} completed")

# Save combined results
output_csv = f'{base_path}CombinedOutput_SLF_Struct_4comps.csv'
combined_df.to_csv(output_csv, index=False)

print(f"\n{'='*60}")
print("STRUCTURAL COMPONENTS SLF GENERATION COMPLETED")
print(f"{'='*60}")
print(f"Processed {len(cases)} cases")
print(f"Combined results saved to: {output_csv}")